In [2]:
import re
from aocd import get_data, submit 
from dataclasses import dataclass

# @dataclass(frozen=True)
# class Money:
#     ore: int = 0
#     clay: int = 0
#     obsidian: int = 0
#     geode: int = 0
        
#     def __add__(self, other):
#         return Money(
#             self.ore + other.ore,
#             self.clay + other.clay,
#             self.obsidian + other.obsidian,
#             self.geode + other.geode)
    
#     def __mul__(self, other):
#         return Money(
#             self.ore * other,
#             self.clay * other,
#             self.obsidian * other,
#             self.geode * other)
    
#     def __rmul__(self, other):
#         return self.__mul__(other)
    
#     def __gt__(self, other):
#         return (
#             (self.ore > other) and
#             (self.clay > other) and
#             (self.obsidian > other) and
#             (self.geode > other) )

# @dataclass(frozen=True)
# class Blueprint:
#     ore_robot: Money
#     clay_robot: Money
#     obsidian_robot: Money
#     geode_robot: Money

# @dataclass(frozen=True)
# class State:
#     money: Money = Money()
#     n_ore_bots: int = 0
#     n_clay_bots: int = 0
#     n_obsidian_bots: int = 0
#     n_geode_bots: int = 0
        
# def produce(s: State) -> State:
#         m = s.money + Money(
#             s.n_ore_bots, 
#             s.n_clay_bots, 
#             s.n_obsidian_bots, 
#             s.n_geode_bots)
#         return State(m, s.n_ore_bots, s.n_clay_bots, s.n_obsidian_bots, s.n_geode_bots)
    
# data = get_data(year=2022, day=19)
# data = """Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
# Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."""
# data = data.split("\n")
# data = [list(map(int, re.findall(r"( [\d]* )", line))) for line in data]
# data = [Blueprint(
#     Money(ore=d[0]),
#     Money(ore=d[1]),
#     Money(ore=d[2], clay=d[3]),
#     Money(ore=d[4], obsidian=d[5])
#         ) for d in data]

# data

In [76]:
import numpy as np
data = get_data(year=2022, day=19)
data = """Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."""
data = data.split("\n")
data = [list(map(int, re.findall(r"( [\d]* )", line))) for line in data]
data = [np.array((
    np.array((d[0], 0, 0, 0)),
    np.array((d[1], 0, 0, 0)),
    np.array((d[2], d[3], 0, 0)),
    np.array((d[4], 0, d[5], 0)))) for d in data]

bp = data[0]
bp


array([[ 4,  0,  0,  0],
       [ 2,  0,  0,  0],
       [ 3, 14,  0,  0],
       [ 2,  0,  7,  0]])

In [77]:
start_bots = np.array((1, 0, 0, 0))
start_money = np.array((0, 0, 0, 0))
state = (start_bots, start_money)
to_set = lambda s: (tuple(s[0]), tuple(s[1]))
from_set = lambda s: (np.array(s[0]), np.array(s[1]))
states = set([to_set(state)])
states

{((1, 0, 0, 0), (0, 0, 0, 0))}

In [78]:
to_set(state)

((1, 0, 0, 0), (0, 0, 0, 0))

In [107]:
def buy_options(bp, m):
    return (np.min(money - bp, axis=1) >= 0)

In [115]:
def find_best_moneys(moneys):
    moneys.sort(reverse=True)
    best = set()
    new_set = set()
    for item in moneys:
        item = np.array(item)
        best = new_set
        new_set = best.copy()
        for b in best:
            b = np.array(b)
            if np.all(b - item >= 0):
                break
        else:
            new_set.add(tuple(item))
            for b in best:
                b = np.array(b)
                if(np.all(item - b >= 0)):
                    new_set.remove(tuple(b))
        best = new_set
    return new_set
            

In [116]:
from itertools import groupby
from collections import defaultdict

# def reduce_states(states):
#     states = sorted(list(states))
#     output = set()
#     for bots, m_gen in groupby(states, lambda s: s[0]):
#         moneys = [m[1] for m in m_gen]
#         best_moneys = find_best_moneys(moneys)
# #         print(bots, moneys)
# #         print(bots, best_moneys)
#         for bmoney in best_moneys:
#             output.add((bots, bmoney))
#     return output

def reduce_states(states):
    states_dict = defaultdict(list)
    for bots, moneys in states:
        states_dict[bots].append(moneys)
    output = set()
    for bots, moneys in states_dict.items():
        best_moneys = find_best_moneys(moneys)
        for bmoney in best_moneys:
            output.add((bots, bmoney))
    return output

In [117]:
bp

array([[ 4,  0,  0,  0],
       [ 2,  0,  0,  0],
       [ 3, 14,  0,  0],
       [ 2,  0,  7,  0]])

In [7]:
all_nonnegative = lambda x: (x[0] >= 0) and (x[1] >= 0) and (x[2] >= 0) and (x[3] >= 0)

def find_max_geodes(bp):
    can_buy_all = np.max(bp, axis=0)
    states = set([
        ((1, 0, 0, 0), (0, 0, 0, 0))
    ])
    for i in range(10):
        new_states = set()
        for state in states: 
            state = from_set(state)
            bots, money = state
            options = buy_options(bp, money)
            money += bots
            # if we can buy all types, we should at least buy one
            if all_nonnegative(money - can_buy_all):
                new_money = []
                new_bots = []
            else:
                new_money = [money]
                new_bots = [bots]
            for idx, option in enumerate(options):
                if option: 
                    new_money.append(money - bp[idx,:])
                    b = bots.copy()
                    b[idx] += 1
                    new_bots.append(b)
            new_states.update([to_set((b,m)) for b, m in zip(new_bots, new_money)])
        states = new_states
        print(f"{i}: {len(states)} state{'s' if len(states) > 1 else ''}")
        if i < 20:
            states = reduce_states(states)
            print(f"{i}: {len(states)} state{'s' if len(states) > 1 else ''} after reduction")

    geodes = [s[1][3] for s in states]
    return max(geodes)


In [119]:
from datetime import datetime
print(datetime.now())
import cProfile

with cProfile.Profile() as pr:
    find_max_geodes(data[1])
    pr.print_stats()


2022-12-19 23:42:14.356001
0: 4 states
0: 4 states after reduction
1: 16 states
1: 13 states after reduction
2: 52 states
2: 40 states after reduction
3: 157 states
3: 118 states after reduction
4: 454 states
4: 334 states after reduction
5: 1229 states
5: 884 states after reduction
6: 3092 states
6: 2147 states after reduction
7: 7142 states
7: 4796 states after reduction
8: 15297 states
8: 9948 states after reduction
9: 30573 states
9: 19359 states after reduction
         54105043 function calls in 173.199 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000   52.345    0.052  171.776    0.172 1077583975.py:1(find_best_moneys)
    18285    0.100    0.000    0.257    0.000 1902926624.py:1(buy_options)
    73140    0.128    0.000    0.128    0.000 2325535112.py:4(<lambda>)
    18285    0.013    0.000    0.046    0.000 2325535112.py:5(<lambda>)
        1    0.353    0.353  173.195  173.195 3484823619.py:1(find_max_geod

In [91]:
[s for s in states if s[1] == (6, 41, 8, 9)]

[((1, 4, 2, 2), (6, 41, 8, 9))]

In [3]:
import numpy as np 
x = np.array([0,1,2,3])

In [4]:
%timeit np.all(x > 0)

4.87 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [6]:
%timeit all_positive(x)

178 ns ± 0.784 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
